## Imports

In [1]:
import os
import sys
from pathlib import Path

sys.path.extend(['/Users/fabianloewe/Projects/master-thesis/tools/aletheia'])

import pandas as pd
import aletheialib.attacks as attacks
import aletheialib.options.tools as tools

## Prepare data

In [2]:
EMBEDDING_METHODS = [
    'MobiStego', 'Passlok', 'Pictograph', 'PixelKnot', 'PocketStego', 'SteganographyM'
]

BASE_PATH = Path('../datasets/StegoAppDB_stegos_20240309-030352')

df = pd.read_csv(BASE_PATH / 'StegoAppDB_stegos_20240309-030352_stego_directory.csv')
dfs = {m: df[df['embedding_method'] == m] for m in EMBEDDING_METHODS}
dfs_image_pairs = {
    m: list(
        (BASE_PATH / 'covers' / ci, BASE_PATH / 'stegos' / si)
        for ci, si in zip(dfs[m]['cover_image_filename'], dfs[m]['image_filename'])
    )
    for m in EMBEDDING_METHODS
}

In [3]:
pd.DataFrame(((m, len(pairs)) for m, pairs in dfs_image_pairs.items()), columns=['Method', 'Count'])

,Method,Count
0,MobiStego,3060
1,Passlok,1530
2,Pictograph,4800
3,PixelKnot,3060
4,PocketStego,3060
5,SteganographyM,3060


## Create metadata diffs

In [5]:
import traceback


def _merge_dicts(first, second):
    new_dict = {}
    for key in first.keys() | second.keys():
        if key in first and key in second and first[key][1] == second[key][1]:
            new_dict[key] = first[key]
        elif key in first and key not in second:
            new_dict[key] = first[key]
        elif key in second and key not in first:
            new_dict[key] = second[key]
    return new_dict


results = []
for method, image_pairs in dfs_image_pairs.items():
    #if method != 'Pictograph':
    #    continue
    
    print(method)
    try:
        intermediate = attacks.metadata_diff(image_pairs[0][0], image_pairs[0][1])
    except Exception as e:
        intermediate = {}

    for cover, stego in image_pairs[1:]:
        try:
            diff = attacks.metadata_diff(cover, stego)
            #print(stego, '->', diff)
            intermediate = _merge_dicts(intermediate, diff)
            #results.append({'Method': method, **diff})
        except Exception:
            print(traceback.format_exc())
    #intermediate = {k: v[1] for k, v in intermediate.items() if k != 'Size'}
    results.append({'Method': method, **intermediate})
    
results_df = pd.DataFrame(results)
results_df

MobiStego
Passlok
Pictograph
PixelKnot
PocketStego
SteganographyM


,Method,PNG:SignificantBits,File:Directory,PNG:ColorType,File:FileInodeChangeDate,File:FileModifyDate
0,MobiStego,"(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)",NaN,NaN
1,Passlok,NaN,(../datasets/StegoAppDB_stegos_20240309-030352...,NaN,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...",NaN
2,Pictograph,NaN,(../datasets/StegoAppDB_stegos_20240309-030352...,"(0, 2)",NaN,"(2024:03:09 04:31:54+01:00, 2024:03:09 03:09:1..."
3,PixelKnot,NaN,(../datasets/StegoAppDB_stegos_20240309-030352...,NaN,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(2024:03:09 04:21:24+01:00, 2024:03:09 04:02:0..."
4,PocketStego,NaN,(../datasets/StegoAppDB_stegos_20240309-030352...,NaN,NaN,NaN
5,SteganographyM,NaN,(../datasets/StegoAppDB_stegos_20240309-030352...,NaN,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...",NaN


In [12]:
__diffs = []
for cover, stego in dfs_image_pairs['MobiStego'][:100]:
    try:
        diff = attacks.metadata_diff(cover, stego)
        __diffs.append(diff)
    except Exception:
        print(traceback.format_exc())
pd.DataFrame(__diffs)

,File:FileInodeChangeDate,File:FileSize,File:FileModifyDate,File:Directory,File:FileAccessDate,File:FileName,PNG:SignificantBits,SourceFile,PNG:ColorType
0,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(289351, 313075)","(2024:03:09 04:21:34+01:00, 2024:03:09 03:31:1...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:28:55+01:00, 2024:03:21 08:56:2...","(260507.PNG, 260508.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
1,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(289351, 326445)","(2024:03:09 04:21:34+01:00, 2024:03:09 03:17:4...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:28:55+01:00, 2024:03:21 08:56:3...","(260507.PNG, 260509.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
2,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(289351, 339390)","(2024:03:09 04:21:34+01:00, 2024:03:09 04:09:3...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:28:55+01:00, 2024:03:20 23:35:2...","(260507.PNG, 260510.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
3,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(289351, 352487)","(2024:03:09 04:21:34+01:00, 2024:03:09 03:06:3...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:28:55+01:00, 2024:03:20 23:35:3...","(260507.PNG, 260511.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
4,"(2024:04:29 07:46:50+02:00, 2024:04:29 07:46:5...","(289351, 366049)","(2024:03:09 04:21:34+01:00, 2024:03:09 03:20:4...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:28:55+01:00, 2024:03:20 23:27:4...","(260507.PNG, 260512.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
...,...,...,...,...,...,...,...,...,...
95,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...","(223370, 230360)","(2024:03:09 04:20:30+01:00, 2024:03:09 04:16:2...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:39:59+01:00, 2024:03:21 09:08:1...","(267201.PNG, 267202.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
96,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...","(223370, 243664)","(2024:03:09 04:20:30+01:00, 2024:03:09 03:53:0...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:39:59+01:00, 2024:03:21 09:10:4...","(267201.PNG, 267203.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
97,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...","(223370, 255537)","(2024:03:09 04:20:30+01:00, 2024:03:09 03:49:0...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:39:59+01:00, 2024:03:21 09:21:2...","(267201.PNG, 267204.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
98,"(2024:04:29 07:46:49+02:00, 2024:04:29 07:46:5...","(223370, 269098)","(2024:03:09 04:20:30+01:00, 2024:03:09 03:29:2...",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2024:03:21 11:39:59+01:00, 2024:03:21 09:21:2...","(267201.PNG, 267205.PNG)","(8 8 8, 8 8 8 8)",(../datasets/StegoAppDB_stegos_20240309-030352...,"(2, 6)"
